In [103]:
import cv2  as cv
import numpy as np

class EyeMotionDetector:
    
    left_counter=0 
    right_counter=0 
    closing_counter=0
    center_counter = 0
    closing_th = 22
    th_value=5
    face_cascade = cv.CascadeClassifier(r'C:\Users\asema\Desktop\haarcascade_frontalface_default.xml')
    eye_cascade = cv.CascadeClassifier(r"C:\Users\asema\Desktop\haarcascade_eye_tree_eyeglasses.xml")
    
    def detect_faces(self, img):
        gray_frame = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        coords = self.face_cascade.detectMultiScale(gray_frame, 1.3, 5)
        if len(coords) > 1:
            biggest = (0, 0, 0, 0)
            for i in coords:
                if i[3] > biggest[3]:
                    biggest = i
            biggest = np.array([i], np.int32)
        elif len(coords) == 1:
            biggest = coords
        else:
            return None
        for (x, y, w, h) in biggest:
            frame = img[y:y + h, x:x + w]
        return frame
    
    def detect_eyes(self, img):
        img = self.detect_faces(img)
        eyes = self.eye_cascade.detectMultiScale(img, 1.3, 5) 
        width = np.size(img, 1) 
        height = np.size(img, 0)
        left_eye, right_eye = None, None
        for (x, y, w, h) in eyes:
            if y > height / 2:
                pass
            eyecenter = x + w / 2 
            if eyecenter < width * 0.5:
                right_eye = img[y:y + h, x:x + w]
            else:
                left_eye = img[y:y + h, x:x + w]
        return left_eye, right_eye
    
    
    def thresholding(self, value, thresh ):
        if abs(value - thresh) < 1:
            print("CENTER")
            self.left_counter=0
            self.right_counter=0
            return
        if (value<=thresh):  
            self.left_counter=self.left_counter+1 
            self.right_counter = 0
            if (self.left_counter>self.th_value): 
                print ('RIGHT')  
                self.left_counter=0  
        elif(value>=thresh):  
            self.right_counter=self.right_counter+1
            self.left_counter = 0
            if(self.right_counter>self.th_value):
                print ('LEFT')
                self.right_counter=0
    
    def run_detection(self):
        cap = cv.VideoCapture(0)
        while True:
            ret, frame = cap.read()
            if ret is False:
                break

            left_eye, right_eye = None, None
            try:
                left_eye, right_eye = self.detect_eyes(frame)
            except :
                pass
            if self.closing_counter > self.closing_th:
                self.closing_counter = 0
                print("EYES BLINK")
                continue
            if (left_eye is None) or (right_eye is None):
                self.closing_counter += 1
                continue
            img = right_eye
            img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            img = cv.equalizeHist(img)
            ret,thresh = cv.threshold(img, 82, 255, cv.THRESH_BINARY)
            
            circles = cv.HoughCircles(img ,cv.HOUGH_GRADIENT,1,20,param1=90,param2=10,minRadius=8,maxRadius=12)
            if circles is not None:
                circles = np.round(circles[0, :]).astype("int") 
                for (x,y,r) in circles:
                    pix = thresh[x-r:x+r,y]
                    temp = pix 
                    
                    # print("pix value " , pix)
                    black =  pix[pix==0]
                    if len(black) < (7/10 *len(pix)):
                        # print(len(black) /len(pix) )
                        # cv.waitKey(0)
                        continue

                    
                    cv.circle(img, (x, y), r, (0, 255, 0), 2)
                    cv.line(img,(x-r,y),(x+r,y),(255,255,255),thickness=3)
                    self.thresholding(x, int(img.shape[0]/2))
                    # k = cv.waitKey(0)
                    # if k == 27:
                    #     break
                    break
            cv.imshow('image',img)
            cv.imshow("thresh", thresh)
            key = cv.waitKey(30)
            if key == 27:
                break
        cap.release()
        cv.destroyAllWindows()

In [104]:
emd = EyeMotionDetector()

In [105]:
emd.run_detection()

CENTER
LEFT
CENTER
CENTER
LEFT
CENTER
CENTER
CENTER
CENTER
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
EYES BLINK
EYES BLINK
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
EYES BLINK
EYES BLINK
CENTER
CENTER
CENTER
CENTER
LEFT
CENTER
CENTER
CENTER
CENTER
CENTER
CENTER
EYES BLINK
EYES BLINK
CENTER
EYES BLINK
